<a href="https://colab.research.google.com/github/djrl10/trying/blob/master/cobaFakeNewsClassifierUsingLSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Fake News Classifier Using LSTM

Dataset: https://www.kaggle.com/c/fake-news/data#

# New Section

In [0]:
import pandas as pd

In [0]:
df= pd.read_csv('train.csv')

In [0]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [0]:
###Drop Nan Values
df=df.dropna()


In [0]:
## Get the Independent Features

X=df.drop('label',axis=1)

In [0]:
## Get the Dependent features
y=df['label']

In [0]:
X.shape

(18285, 4)

In [0]:
y.shape

(18285,)

In [0]:
import tensorflow as tf

In [0]:
tf.__version__

'2.2.0'

In [0]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [0]:
### Vocabulary size
voc_size=5000

### Onehot Representation

In [0]:
messages=X.copy()

In [0]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [0]:
messages.reset_index(inplace=True)

In [0]:
import nltk
import re
from nltk.corpus import stopwords

In [0]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [0]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    print(i)
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

Streaming output truncated to the last 5000 lines.
13286
13287
13288
13289
13290
13291
13292
13293
13294
13295
13296
13297
13298
13299
13300
13301
13302
13303
13304
13305
13306
13307
13308
13309
13310
13311
13312
13313
13314
13315
13316
13317
13318
13319
13320
13321
13322
13323
13324
13325
13326
13327
13328
13329
13330
13331
13332
13333
13334
13335
13336
13337
13338
13339
13340
13341
13342
13343
13344
13345
13346
13347
13348
13349
13350
13351
13352
13353
13354
13355
13356
13357
13358
13359
13360
13361
13362
13363
13364
13365
13366
13367
13368
13369
13370
13371
13372
13373
13374
13375
13376
13377
13378
13379
13380
13381
13382
13383
13384
13385
13386
13387
13388
13389
13390
13391
13392
13393
13394
13395
13396
13397
13398
13399
13400
13401
13402
13403
13404
13405
13406
13407
13408
13409
13410
13411
13412
13413
13414
13415
13416
13417
13418
13419
13420
13421
13422
13423
13424
13425
13426
13427
13428
13429
13430
13431
13432
13433
13434
13435
13436
13437
13438
13439
13440
13441
13442
13443
1

In [0]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [0]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr

[[557, 4956, 288, 213, 3598, 226, 4274, 1998, 4141, 4921],
 [4902, 4484, 4958, 2170, 220, 349, 4721],
 [4489, 2746, 2744, 4577],
 [3863, 990, 648, 2206, 1045, 6],
 [3185, 220, 311, 4726, 216, 1737, 220, 4345, 3633, 243],
 [3130,
  496,
  3486,
  2201,
  4675,
  1822,
  1019,
  2031,
  3282,
  1875,
  1690,
  3442,
  1501,
  3424,
  4721],
 [1957, 850, 3615, 3071, 96, 4462, 191, 1777, 330, 581, 4221],
 [2561, 2031, 1339, 2019, 2438, 1972, 1822, 4966, 330, 581, 4221],
 [505, 770, 3476, 3018, 1284, 4736, 1174, 2775, 1822, 3751],
 [2527, 916, 2154, 2997, 1402, 1262, 395, 946],
 [3339, 3404, 4421, 165, 3095, 1243, 4021, 1501, 4227, 3716, 189],
 [2206, 2903, 3598, 4736, 1822, 2438],
 [2860, 2489, 4028, 819, 3559, 3557, 2154, 1904, 1274],
 [2203, 4581, 850, 3610, 3632, 38, 1024, 330, 581, 4221],
 [1673, 2237, 4442, 3197, 1634, 330, 581, 4221],
 [4817, 3226, 2256, 3201, 2447, 2655, 226, 2613, 822, 4876],
 [70, 4292, 4484],
 [1423, 1802, 2153, 1290, 1822, 4109, 2046, 4721],
 [4016, 856, 4958, 2

### Embedding Representation

In [0]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 1998 4141 4921]
 [   0    0    0 ...  220  349 4721]
 [   0    0    0 ... 2746 2744 4577]
 ...
 [   0    0    0 ...  330  581 4221]
 [   0    0    0 ...  969  566 2883]
 [   0    0    0 ... 4564  118 2504]]


In [0]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,  557,
       4956,  288,  213, 3598,  226, 4274, 1998, 4141, 4921], dtype=int32)

In [0]:
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 40)            200000    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               56400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [0]:
len(embedded_docs),y.shape

(18285, (18285,))

In [0]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [0]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

### Model Training

In [0]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 6s 30ms/step - loss: 0.3245 - accuracy: 0.8437 - val_loss: 0.1917 - val_accuracy: 0.9170
Epoch 2/10
192/192 [==============================] - 6s 29ms/step - loss: 0.1353 - accuracy: 0.9473 - val_loss: 0.1971 - val_accuracy: 0.9215
Epoch 3/10
192/192 [==============================] - 6s 29ms/step - loss: 0.0950 - accuracy: 0.9656 - val_loss: 0.2291 - val_accuracy: 0.9210
Epoch 4/10
192/192 [==============================] - 6s 29ms/step - loss: 0.0733 - accuracy: 0.9744 - val_loss: 0.2456 - val_accuracy: 0.9082
Epoch 5/10
192/192 [==============================] - 5s 28ms/step - loss: 0.0537 - accuracy: 0.9829 - val_loss: 0.2992 - val_accuracy: 0.9118
Epoch 6/10
192/192 [==============================] - 6s 29ms/step - loss: 0.0372 - accuracy: 0.9874 - val_loss: 0.3580 - val_accuracy: 0.9167
Epoch 7/10
192/192 [==============================] - 6s 29ms/step - loss: 0.0223 - accuracy: 0.9923 - val_loss: 0.3641 - val_accuracy: 0.9099

### Adding Dropout 

In [0]:
from tensorflow.keras.layers import Dropout
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

### Performance Metrics And Accuracy

In [0]:
y_pred=model.predict_classes(X_test)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


In [0]:
from sklearn.metrics import confusion_matrix

In [0]:
confusion_matrix(y_test,y_pred)

array([[1111, 2308],
       [1200, 1416]])

In [0]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.4187241093620547